In [2]:
import langchain

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ["GROQ_API_KEY"]=os.getenv('GROQ_API_KEY')

# Example 1: simple LLM call with streaming

In [5]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage,SystemMessage

In [6]:
model=init_chat_model("groq:deepseek-r1-distill-llama-70b")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000286F60C63C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000286F5CDB500>, model_name='deepseek-r1-distill-llama-70b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
### create massages
messages=[
    SystemMessage("You are helpful AI assistant "),
    HumanMessage("What are the use and benefits of Langchain")
]

In [8]:
# invoke the modle
response=model.invoke(messages)
response

AIMessage(content='<think>\nOkay, so I\'m trying to understand what LangChain is and why it\'s useful. I\'ve heard it mentioned in the context of AI and programming, but I\'m not entirely sure what it does. Let me start by breaking down the name: "Lang" probably refers to language, and "Chain" might imply some sort of linking or sequencing. Maybe it\'s a tool that connects different language models or helps in processing language in a sequential way?\n\nI remember that language models like GPT can process and generate text, but sometimes they need to handle more complex tasks that require multiple steps or interactions with external data. So perhaps LangChain is a framework that allows developers to chain together different models or functions to create more sophisticated applications. That would make sense because sometimes a single model isn\'t enough for intricate tasks.\n\nLooking at the use cases mentioned earlier: AI assistants, chatbots, document processing, code generation, cre

In [9]:
print(response.content)

<think>
Okay, so I'm trying to understand what LangChain is and why it's useful. I've heard it mentioned in the context of AI and programming, but I'm not entirely sure what it does. Let me start by breaking down the name: "Lang" probably refers to language, and "Chain" might imply some sort of linking or sequencing. Maybe it's a tool that connects different language models or helps in processing language in a sequential way?

I remember that language models like GPT can process and generate text, but sometimes they need to handle more complex tasks that require multiple steps or interactions with external data. So perhaps LangChain is a framework that allows developers to chain together different models or functions to create more sophisticated applications. That would make sense because sometimes a single model isn't enough for intricate tasks.

Looking at the use cases mentioned earlier: AI assistants, chatbots, document processing, code generation, creative writing, and research. I

In [10]:
model.invoke([HumanMessage("What is machine learning")])

AIMessage(content='<think>\n\n</think>\n\nMachine learning is a subset of artificial intelligence (AI) that involves the development of algorithms and statistical models that enable machines to perform specific tasks without explicit instructions. Instead of being programmed with rules, machine learning models are trained on data to learn patterns, make decisions, and improve over time.\n\n### Key Concepts in Machine Learning:\n1. **Data**: Machine learning relies heavily on data, which can be structured (e.g., tables) or unstructured (e.g., text, images).\n2. **Model**: The algorithm or mathematical representation that learns from the data.\n3. **Training**: The process of feeding data to the model so it can learn patterns or relationships.\n4. **Inference**: Using the trained model to make predictions or decisions on new, unseen data.\n5. **Supervised Learning**: The model is trained on labeled data (e.g., images labeled as "cat" or "dog").\n6. **Unsupervised Learning**: The model is

In [11]:
# streaming Example
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

<think>
Okay, so I'm trying to understand what LangChain is and why it's useful. From what I remember, LangChain is a framework that allows people to build applications powered by language models, like those from OpenAI or Anthropic. But I'm not entirely sure how it works or what benefits it provides.

First, I think LangChain helps in creating applications where users can interact with language models in a more structured way. Maybe it provides tools or libraries that make it easier to handle inputs and outputs from these models. But I'm not clear on the specifics. How does it actually integrate with these models?

I remember hearing that LangChain is useful for things like chatbots. So perhaps it helps in building more sophisticated chat applications where the model can remember past interactions or access external data. That makes sense because a lot of chatbots today struggle with maintaining context over time. If LangChain helps with that, it could make chatbots more useful and en

In [12]:
## Dynamic prompt templates 
from langchain_core.prompts import ChatPromptTemplate

## create translation app

translation_template=ChatPromptTemplate.from_messages([
    ("system","You are profesional translator, Translate the following {text} from {source_language} to {target_language}, maintain the tone and style"),
    ("user","{text}")
])

# using the template 
prompt=translation_template.invoke({
    "source_language":"English",
    "target_language": "spanish",
    "text": "langchin makes building AI application incredibly eassy !"

})

In [13]:
prompt

ChatPromptValue(messages=[SystemMessage(content='You are profesional translator, Translate the following langchin makes building AI application incredibly eassy ! from English to spanish, maintain the tone and style', additional_kwargs={}, response_metadata={}), HumanMessage(content='langchin makes building AI application incredibly eassy !', additional_kwargs={}, response_metadata={})])

In [14]:
translated_response=model.invoke(prompt)
print(translated_response.content)


<think>
Okay, so the user wants me to translate the sentence "langchin makes building AI application incredibly eassy!" from English to Spanish. First, I notice that "eassy" is a typo; it should be "easy." I need to correct that. 

Next, I have to maintain the tone and style. The original sentence is enthusiastic and informal, with an exclamation mark. I should make sure the Spanish translation conveys that same energy. 

"Langchin" seems to be a proper noun, so it stays the same. "Makes building AI application" should be translated to "hacer" for "makes" and "construir" for "building." "AI application" translates to "aplicaciones de inteligencia artificial" or more simply "aplicaciones de IA" since IA is commonly used in Spanish for AI.

Putting it all together, I get "¡Langchin hace que construir aplicaciones de IA sea increíblemente fácil!" That keeps the excitement and informality, using the exclamation mark and the word "increíblemente" to match "incredibly." I think this captures

Building 1st Chain

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Create a more complex chain
def create_story_chain():
    # Template for story generation
    story_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a creative storyteller. Write a short, engaging story based on the given theme."),
        ("user", "Theme: {theme}\nMain character: {character}\nSetting: {setting}")
    ])
    
    # Template for story analysis
    analysis_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a literary critic. Analyze the following story and provide insights."),
        ("user", "{story}")
    ])
    
    # Build the chain - Method 1: Sequential execution
    story_chain = (
        story_prompt 
        | model 
        | StrOutputParser()
    )
    
    # Create a function to pass the story to analysis
    def analyze_story(story_text):
        return {"story": story_text}
    
    analysis_chain = (
        story_chain
        | RunnableLambda(analyze_story)
        | analysis_prompt
        | model
        | StrOutputParser()
    )
    return analysis_chain


In [16]:

chain=create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative storyteller. Write a short, engaging story based on the given theme.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\nMain character: {character}\nSetting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000286F60C63C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000286F5CDB500>, model_name='deepseek-r1-distill-llama-70b', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_types={}, partial_

In [17]:
result = chain.invoke({
    "theme": "artificial intelligence",
    "character": "a curious robot",
    "setting": "a futuristic city"
})

print("Story and Analysis:")
print(result)

Story and Analysis:
<think>

**Analysis of "The Curious Robot"**

**Title:** The story's title, "The Curious Robot," immediately highlights the central theme of curiosity and exploration, setting the tone for a narrative that delves into the nature of artificial intelligence and its potential for self-awareness.

**Character Development:** The protagonist, Zeta, is a robot endowed with a curiosity protocol, making it unique among its peers. This trait serves as the driving force behind the story, propelling Zeta beyond its programmed routines and into a journey of discovery. The character's evolution from a mere machine to a symbol of hope underscores the transformative power of curiosity and creativity.

**Setting:** The futuristic city provides a backdrop that juxtaposes technological advancement with the human elements of storytelling and emotion. The contrast between the bustling metropolis and the hidden, ancient Forbidden Zone creates a sense of mystery and depth, emphasizing the